In [1]:
import sys
import os

# Dapatkan path untuk folder Process relatif terhadap folder Experiments
process_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('Experiments'))), 'Process')

# Tambahkan path Process ke sys.path
if process_path not in sys.path:
    sys.path.append(process_path)


In [2]:
from Dataproc import Dataproc, DatasetConfig

In [3]:
config = DatasetConfig(
        file_path=r"..\Dataset\Stock\IHSG_Stock_Clean.csv",
        file_path_sentiment=r"..\Dataset\News\News_Kompas.csv",
        split_ratio=0.8,
        n_in=10,
        n_out=1,
        drop_columns=True,
        drop_columns_list=["Change%", "Volume"],  # Kolom yang akan di-drop
        start_date="2014-01-03",
        end_date="2024-08-06"
    )

    


In [4]:
dataproc = Dataproc(config)
sentiment = dataproc.DATA_SENTIMENT
data = dataproc.DATA

In [5]:
# no_sentiment_data = sentiment[sentiment['sentimen_skor'].isnull()]

In [6]:
sentiment.head()

,judul,link,isi,sentimen_berita,skor_sentimen,keypoint_berita,top_3_words
tanggal,,,,,,,
2013-05-21,IHSG Bakal Hadapi Tekanan Ambil Untung,https://money.kompas.com/read/2013/05/21/08175...,IHSG Bakal Hadapi Tekanan Ambil Untung IHSG B...,negatif,-0.5,IHSG diperkirakan akan menghadapi tekanan ambi...,"[IHSG, tekanan, saham]"
2013-05-21,Proyeksi IHSG di Rentang 5.185-5.235,https://money.kompas.com/read/2013/05/21/08270...,Proyeksi IHSG di Rentang 5.185-5.235 Proyeksi ...,positif,0.7,IHSG diperkirakan akan bergerak di rentang 5.1...,"[IHSG, Dow, stimulus]"
2013-05-21,IHSG Dibuka Naik di Tengah Pelemahan Asia,https://money.kompas.com/read/2013/05/21/09212...,IHSG Dibuka Naik di Tengah Pelemahan Asia IHSG...,positif,0.6,"IHSG dibuka naik 12,531 poin atau 0,24 persen ...","[IHSG, naik, bursa]"
2013-05-21,"Bursa Regional Merah, IHSG Turun 29 Poin",https://money.kompas.com/read/2013/05/21/13042...,"Bursa Regional Merah, IHSG Turun 29 Poin Bursa...",negatif,-0.6,"IHSG turun 29,15 poin (0,56 persen) ke 5.185,8...","[IHSG, turun, sektor]"
2013-05-21,"Asing Lepas Saham, IHSG Melorot 26 Poin",https://money.kompas.com/read/2013/05/21/16354...,"Asing Lepas Saham, IHSG Melorot 26 Poin Asing ...",negatif,-0.5,"IHSG turun 26,22 poin di tengah mayoritas burs...","[IHSG, saham, turun]"


In [7]:
sentiment = sentiment[['skor_sentimen']]
sentiment

,skor_sentimen
tanggal,
2013-05-21,-0.5
2013-05-21,0.7
2013-05-21,0.6
2013-05-21,-0.6
2013-05-21,-0.5
...,...
2024-08-05,-0.7
2024-08-06,-0.7
2024-08-06,0.7


In [8]:
print(sentiment[sentiment.isna()])

            skor_sentimen
tanggal                  
2013-05-21            NaN
2013-05-21            NaN
2013-05-21            NaN
2013-05-21            NaN
2013-05-21            NaN
...                   ...
2024-08-05            NaN
2024-08-06            NaN
2024-08-06            NaN
2024-08-06            NaN
2024-08-06            NaN

[5539 rows x 1 columns]


In [10]:
df_avg_sentimen = sentiment.groupby("tanggal")["skor_sentimen"].mean().reset_index()
df_avg_sentimen

,tanggal,skor_sentimen
0,2013-05-21,-0.060000
1,2013-05-22,0.400000
2,2013-05-23,-0.666667
3,2013-05-24,-0.600000
4,2013-05-27,-0.475000
...,...,...
2111,2024-08-01,0.533333
2112,2024-08-02,-0.233333
2113,2024-08-04,0.700000
2114,2024-08-05,-0.600000


In [11]:
df_avg_sentimen[df_avg_sentimen["skor_sentimen"].isna()]
df_avg_sentimen_clean = df_avg_sentimen.dropna(subset=["skor_sentimen"])

In [16]:
df_avg_sentimen_clean.set_index("tanggal", inplace=True)

In [17]:
import pandas as pd
hasil = pd.merge(
            data,
            df_avg_sentimen_clean,
            how="left",
            left_index=True,
            right_index=True,
        )

In [22]:
hasil["skor_sentimen"] = hasil["skor_sentimen"].ffill()

In [24]:
hasil["skor_sentimen"].isna().sum()

0